In [ ]:
%cd ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

In [ ]:
real_500_df = pd.read_csv('data/out/sample.real-500.csv', index_col=0)
real_500_df

In [ ]:
syn_500_df = pd.read_csv('data/out/sample.syn-500.csv', index_col=0).drop('n_tokens', axis=1)
syn_500_df

In [ ]:
syn_1000_df = pd.read_csv('data/out/sample.syn-1000.csv', index_col=0).drop('n_tokens', axis=1)
syn_1000_df

In [ ]:
from src.data_ingestion import *

real_1000_df = get_stratified_data('data/train_pool.csv', 1_000, 42)
real_2000_df = get_stratified_data('data/train_pool.csv', 2_000, 42)

In [ ]:
test_df = pd.read_csv('data/test.csv')
test_df

In [ ]:
from src.model_evaluation import *
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier()
model_eval = ModelEvaluator(clf, test_df['text'], test_df['label'])

In [ ]:
model_eval.evaluate(real_500_df['text'], real_500_df['label'])

In [ ]:
model_eval.evaluate(real_1000_df['text'], real_1000_df['label'])

In [ ]:
model_eval.evaluate(real_2000_df['text'], real_2000_df['label'])

In [ ]:
real_500_syn_500_df = pd.concat([real_500_df, syn_500_df], ignore_index=True)
real_500_syn_500_df

In [ ]:
model_eval.evaluate(real_500_syn_500_df['text'], real_500_syn_500_df['label'])

In [ ]:
real_500_syn_1000_df = pd.concat([real_500_df, syn_1000_df], ignore_index=True)
real_500_syn_1000_df

In [ ]:
model_eval.evaluate(real_500_syn_1000_df['text'], real_500_syn_1000_df['label'])

In [ ]:
real_500_syn_1500_df = pd.concat([real_500_df, syn_500_df, syn_1000_df], ignore_index=True)
real_500_syn_1500_df

In [ ]:
model_eval.evaluate(real_500_syn_1500_df['text'], real_500_syn_1500_df['label'])